# LlamBERT: Large-scale low-cost data annotation in NLP

## Proyecto final.  Procesamiento del Lenguaje Natural

Guillermo Segura Gómez

### Introducción

En la actualidad cuando nos referimos a la tarea de etiquetar un corpus de lenguaje natural los Grandes Modelos de Lenguaje (LLMs) como GPT-4 y Llama 2 se presentan a si mismos como una muy buena solución. De hecho un mínimo de *prompt-tuning* es suficiente para ser altamente competentes en muchas tareas de NLP. Sin embargo correr millones de prompts demanda una gran cantidad de recursos computacionales. Una muy buena alternativa resulta en la combinación de un LLM con modelos mas pequeños pero muy eficientes.

El artículo plantea una metodología híbrida denominada **LlamBERT** en la que se utiliza un LLM, en este caso Llama 2, para etiquetar un subconjunto de datos extraídos aleatoriamente del corpus. Con estas etiquetas se realiza un fine tuning a un modelo tipo transformer encoder de pequeña escala para poder etiquetar el corpus completo. Esto plantea una reducción bastante significativa del *tiempo de inferencia* además de la reducción del costo computacional, mientras se mantiene una alta precisión.

### Enfoque metodológico
* Pasos del Enfoque LlamBERT:
  1. Anotación Inicial: Utiliza Llama-2 para etiquetar un subconjunto aleatorio de datos no etiquetados con una configuración de 0-shot.
  2. Clasificación: Clasifica las respuestas de Llama-2 en las categorías deseadas.
  3. Filtrado: Descarta los datos que no se clasifican en ninguna categoría especificada.
  4. Fine-Tuning: Utiliza las etiquetas resultantes para afinar un clasificador BERT.
  5. Etiquetado Completo: Aplica el clasificador BERT afinado para anotar el corpus no etiquetado original.

### Datasets Utilizados
* IMDb Dataset: Dataset de reseñas de películas utilizado para evaluar la clasificación de sentimientos. Incluye 25,000 reseñas etiquetadas para entrenamiento, 25,000 para prueba y 50,000 reseñas no etiquetadas.
* UMLS Dataset: Vocabulario biomédico utilizado para evaluar la clasificación de conceptos anatómicos. Incluye 3 millones de conceptos, de los cuales se seleccionaron 150,000 conceptos anatómicos.

### Resultados Experimentales

Primero se realizó una prueba utilizando el dataset IMDb con los LLM realizando la tarea de etiquetado completo.

#### Tabla 1: Comparación de Rendimiento en IMDb


**Objetivo:**
Comparar el rendimiento de los modelos Llama-2 y GPT-4 en la tarea de clasificación de sentimientos en el conjunto de datos IMDb utilizando diferentes configuraciones de few-shot (0-shot, 1-shot y 2-shot).

**Modelos Comparados:**
1. **Llama-2-7b-chat:** Un modelo más pequeño de Llama-2.
2. **Llama-2-70b-chat:** Un modelo más grande y potente de Llama-2.
3. **GPT-4-0613:** Modelo de OpenAI, evaluado solo en configuración 0-shot debido a las limitaciones de acceso a la API.

**Configuraciones de Few-Shot:**
- **0-shot:** El modelo no recibe ejemplos específicos de la tarea antes de realizar la predicción.
- **1-shot:** El modelo recibe un ejemplo específico de la tarea antes de realizar la predicción.
- **2-shot:** El modelo recibe dos ejemplos específicos de la tarea antes de realizar la predicción.


**Interpretación de los Resultados:**
- **Mejora con Few-Shot:** Para Llama-2-7b-chat, la precisión mejora significativamente al pasar de 0-shot a 2-shot, lo que indica que el modelo se beneficia mucho de los ejemplos adicionales.
- **Estabilidad en Llama-2-70b-chat:** Para el modelo más grande Llama-2-70b-chat, la precisión es alta incluso en la configuración 0-shot, y no mejora mucho con ejemplos adicionales, lo que sugiere que ya es muy capaz sin necesidad de ejemplos específicos.
- **Rendimiento de GPT-4:** El modelo GPT-4 tiene una alta precisión en 0-shot, lo que destaca su capacidad en la tarea sin necesidad de entrenamiento adicional.

**Tiempos de Inferencia:**
- **Llama-2-7b-chat:** Más rápido que Llama-2-70b-chat debido a su menor tamaño.
- **Llama-2-70b-chat:** Tiempos de inferencia significativamente más largos debido a su tamaño y complejidad.
- **GPT-4:** No se proporcionaron tiempos de inferencia específicos, pero generalmente se sabe que modelos grandes como GPT-4 tienen tiempos de inferencia largos.

#### Tabla 2: Fine-Tuning de Modelos BERT

**Objetivo:**
Evaluar el rendimiento de diferentes modelos BERT preentrenados afinados con datos estándar y con datos etiquetados por Llama-2-70b-chat.

**Modelos Comparados:**
1. **distilbert-base**
2. **bert-base**
3. **bert-large**
4. **roberta-base**
5. **roberta-large**

**Configuraciones de Datos de Entrenamiento:**
- **Baseline:** Afinado con datos de entrenamiento estándar (gold-standard).
- **LlamBERT:** Afinado con datos etiquetados por Llama-2-70b-chat en configuración 0-shot.
- **LlamBERT con Datos Adicionales:** Afinado con datos etiquetados por Llama-2-70b-chat más 50,000 datos adicionales.
- **Estrategia Combinada:** Primero afinado con datos etiquetados por Llama-2 y luego afinado nuevamente con datos estándar.


**Interpretación de los Resultados:**
- **Precisión de LlamBERT:** Los modelos afinados con etiquetas generadas por Llama-2 (LlamBERT) tienen una precisión similar a los afinados con datos estándar, destacando la efectividad del enfoque LlamBERT.
- **Mejora con Datos Adicionales:** Añadir 50,000 datos adicionales etiquetados por Llama-2 mejoró ligeramente la precisión en todos los modelos.
- **Mejor Desempeño:** RoBERTa-large alcanzó la mejor precisión (96.68%) en la estrategia combinada, mostrando la ventaja de usar tanto etiquetas generadas por Llama como datos estándar.

#### Resumen del Proceso Experimental

1. **Etiquetado Inicial con Llama-2:** Llama-2-70b-chat fue utilizado para etiquetar un subconjunto de datos no etiquetados en una configuración de 0-shot.
2. **Fine-Tuning de BERT:**
   - Modelos BERT preentrenados fueron afinados con estas etiquetas generadas por Llama-2.
   - Se compararon los resultados de estos modelos con los afinados usando datos estándar.
3. **Evaluación y Comparación:**
   - Se evaluó la precisión de los modelos en el conjunto de datos de prueba de IMDb.
   - Se analizaron los tiempos de inferencia y la eficiencia de cada modelo.
4. **Análisis del Error:**
   - Se evaluó el impacto del etiquetado incorrecto y se realizó un análisis manual del error en las predicciones del modelo.


El artículo demuestra que el enfoque LlamBERT es una solución práctica y efectiva para la anotación de datos a gran escala en NLP, combinando la potencia de LLMs como Llama-2 para generar etiquetas con la eficiencia de modelos más pequeños como BERT para el fine-tuning. Esto permite reducir costos computacionales y mantener una alta precisión en tareas específicas.

### Análisis del Error

- **Cantidad de Datos y Precisión:**
  - **Proceso:** Afinaron `roberta-large` usando diferentes tamaños de subconjuntos de datos de entrenamiento estándar y datos etiquetados por Llama-2-70b-chat.
  - **Observación:** La mejora en el rendimiento se estabiliza rápidamente en el caso de LlamBERT, indicando que aumentar la cantidad de datos más allá de cierto punto no mejora significativamente la precisión.
  - **Conclusión:** Etiquetar 10,000 entradas es suficiente para obtener un buen equilibrio entre precisión y eficiencia.
- **Impacto del Etiquetado Incorrecto:**
  - **Experimento:** Compararon el impacto del error de etiquetado de Llama-2 (4.61%) con el etiquetado incorrecto aleatorio.
  - **Resultados:** `roberta-large` muestra resistencia al error aleatorio, pero los errores de Llama-2 tienen un impacto mayor en la precisión.
- **Análisis Manual del Error:**
  - **Proceso:** Revisaron manualmente 100 reseñas mal clasificadas, obteniendo anotaciones humanas independientes.
  - **Resultados:** Las salidas del modelo se alineaban más con el sentimiento humano que con las etiquetas estándar, sugiriendo que las etiquetas humanas podrían ser más precisas en algunos casos.


El artículo muestra cómo el enfoque LlamBERT puede ser una solución efectiva y eficiente para la anotación de grandes volúmenes de datos en NLP. Al combinar LLMs como Llama-2 para la anotación inicial con modelos más pequeños como BERT para la afinación, es posible reducir los costos computacionales sin sacrificar significativamente la precisión.

## Implemtación

Para esta parte se va implementar el artículo, primero etiquetando el corpus utilizando los modelos de Llama-2 y GPT-4 y luego utilizando esos datos para realizar un fine-tuning a un modelo BERT para lograr etiquetar el corpus completo.


In [1]:
# Librerias
!pip install neptune python-dotenv
!pip install --upgrade transformers[torch] accelerate
!pip install --upgrade transformers neptune-client neptune

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 10.4 MB/s eta 0:00:00
  Created wheel for bravado-core: filename=bravado_core-6.1.1-py2.py3-none-any.whl size=67672 sha256=b57e914c44f6a4acb1868059fd326bfa0f0b29c01166de4eb5176406d1bde204
  Stored in directory: /root/.cache/pip/wheels/42/35/4a/44e

In [2]:
!git clone https://github.com/aielte-research/LlamBERT.git

Cloning into 'LlamBERT'...
remote: Enumerating objects: 1309, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1309 (delta 3), reused 1 (delta 0), pack-reused 1297
Receiving objects: 100% (1309/1309), 33.51 MiB | 11.33 MiB/s, done.
Resolving deltas: 100% (800/800), done.
Updating files: 100% (168/168), done.


In [3]:
%cd /content/LlamBERT/BERT

/content/LlamBERT/BERT


In [4]:
!python bert_finetune.py -c conf/UMLS/region_10k_quicktest.yaml

2024-05-28 15:46:03.779419: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 15:46:03.779475: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 15:46:03.882829: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-28 15:46:03.905639: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 15:46:05.140362: W tensorflow/comp

In [ ]:
import torch
torch.cuda.empty_cache()

In [5]:
# Importar librerías
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Especifica la ruta donde se guardó tu modelo y los datos de prueba
model_path = "/content/saved_model"
test_data_path = "data/UMLS_regions_10k/short_prompt/test.json"

# Cargar el tokenizer y el modelo
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Mover el modelo a la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Cargar los datos de prueba
with open(test_data_path, "r") as f:
    test_data = json.load(f)

# Preparar los textos y las etiquetas
texts = [item['txt'] for item in test_data]
labels = [item['label'] for item in test_data]

# Tokenizar y hacer predicciones en lotes más pequeños
batch_size = 8  # Ajusta este valor según sea necesario

predicted_labels = []
model.eval()
with torch.no_grad():
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
        predicted_labels.extend(predictions.cpu().numpy().tolist())

print(predicted_labels)

# Evaluar el rendimiento
accuracy = accuracy_score(labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(labels, predicted_labels, average="binary")

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

100%|██████████| 125/125 [00:25<00:00,  4.89it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
import shutil

# Comprimir la carpeta saved_model
shutil.make_archive('/content/saved_model', 'zip', 'saved_model')


'/content/saved_model.zip'

In [7]:
from google.colab import files

# Descargar el archivo comprimido
files.download('/content/saved_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!cp -r /content/saved_model /content/drive/MyDrive/

## Demo modelo

Predicciones básicas

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Cargar el modelo y el tokenizer
model_path = "/content/saved_model"  # Ajusta esta ruta según tu ubicación
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Configurar el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Definir una función para hacer predicciones
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions.cpu().numpy()[0]

# Ejemplo de uso
text = "This is a groundbreaking discovery in the field of medical science."
prediction = predict(text)
print(f"Prediction: {'Positive' if prediction == 1 else 'Negative'}")

Prediction: Negative


In [17]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import json

# Cargar los datos de prueba
test_data_path = "data/UMLS_regions_10k/short_prompt/test.json"
with open(test_data_path, "r") as f:
    test_data = json.load(f)

# Preparar los textos y las etiquetas
texts = [item['txt'] for item in test_data]
labels = [item['label'] for item in test_data]

# Tokenizar y hacer predicciones en lotes
batch_size = 8
predicted_labels = []
model.eval()
with torch.no_grad():
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
        predicted_labels.extend(predictions.cpu().numpy().tolist())

# Evaluar el rendimiento
accuracy = accuracy_score(labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(labels, predicted_labels, average="binary")

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Accuracy: 0.789
Precision: 0.0
Recall: 0.0
F1 Score: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
# Función para mostrar predicciones con ejemplos específicos
def display_predictions(texts, labels, predictions, num_examples=5):
    for i in range(num_examples):
        print(f"Text: {texts[i]}")
        print(f"Actual Label: {'Negative' if labels[i] == 1 else 'Negative'}")
        print(f"Predicted Label: {'Positive' if predictions[i] == 1 else 'Negative'}")
        print("="*50)

# Mostrar ejemplos específicos
display_predictions(texts, labels, predicted_labels)

Text: Decide if the term: medial internal nasal branch of anterior ethmoidal nerve; medial internal nasal branches of anterior ethmoidal nerve; rami nasales interni mediales nervus ethmoidalis anterioris; set of medial internal nasal branches of anterior ethmoidal nerve is related to the human nervous system. Exclude the only vascular structures, even if connected to the nervous system. If multiple examples or terms with multiple words are given, treat them all as a whole and make your decision based on that.
Actual Label: Negative
Predicted Label: Negative
Text: Decide if the term: brodmann area 11 of right subcallosal gyrus; brodmann area 11 of right paraterminal gyrus is related to the human nervous system. Exclude the only vascular structures, even if connected to the nervous system. If multiple examples or terms with multiple words are given, treat them all as a whole and make your decision based on that.
Actual Label: Negative
Predicted Label: Negative
Text: Decide if the term: t